In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test.fillna('', inplace=True)

In [3]:
train

,label,text,diff
0,negative,Require manage_options or pmpro_ caps to get o...,diff --git a/includes/services.php b/includes/...
1,negative,Removed unused image,diff --git a/.idea/workspace.xml b/.idea/works...
2,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
3,positive,SEC-2500: Prevent anonymous bind for ActiveDir...,diff --git a/core/src/main/resources/org/sprin...
4,negative,HDFS-15604. Fix Typo for HdfsDataNodeAdminGuid...,diff --git a/hadoop-hdfs-project/hadoop-hdfs/s...
...,...,...,...
197,positive,Release v5.0.9; see CHANGELOG for details.,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
198,positive,Reject replies to DNS PTR requests that contai...,diff --git a/src/dns.cpp b/src/dns.cpp\nindex ...
199,negative,drop more waffle flags (#7739)\n\nFixes #7734,diff --git a/docs/feature-toggles.rst b/docs/f...
200,negative,Merge pull request #100 from adisingh007/hapi#...,diff --git a/lib/batch.js b/lib/batch.js\ninde...


In [4]:
train_code_change = list(train['diff'].astype(str))
test_code_change = list(test['diff'].astype(str))

In [5]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [6]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [7]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['positive','negative'], sample_size=8)

In [8]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 218
})

In [9]:
len(train)

202

In [10]:
train['label'].value_counts()

label
negative    127
positive     75
Name: count, dtype: int64

In [11]:
test['label'].value_counts()

label
negative    6220
positive    3690
Name: count, dtype: int64

In [12]:
len(train_dataset)

218

In [13]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [14]:
test['label'].value_counts()

label
negative    6220
positive    3690
Name: count, dtype: int64

In [15]:
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 218
})

In [16]:
train

,label,text,diff
0,negative,Require manage_options or pmpro_ caps to get o...,diff --git a/includes/services.php b/includes/...
1,negative,Removed unused image,diff --git a/.idea/workspace.xml b/.idea/works...
2,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
3,positive,SEC-2500: Prevent anonymous bind for ActiveDir...,diff --git a/core/src/main/resources/org/sprin...
4,negative,HDFS-15604. Fix Typo for HdfsDataNodeAdminGuid...,diff --git a/hadoop-hdfs-project/hadoop-hdfs/s...
...,...,...,...
197,positive,Release v5.0.9; see CHANGELOG for details.,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
198,positive,Reject replies to DNS PTR requests that contai...,diff --git a/src/dns.cpp b/src/dns.cpp\nindex ...
199,negative,drop more waffle flags (#7739)\n\nFixes #7734,diff --git a/docs/feature-toggles.rst b/docs/f...
200,negative,Merge pull request #100 from adisingh007/hapi#...,diff --git a/lib/batch.js b/lib/batch.js\ninde...


In [17]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [18]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [19]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    train_code_change = train_code_change,
    test_code_change = test_code_change,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [20]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 35.83it/s]
***** Running training *****
  Num examples = 8720
  Num epochs = 1
  Total optimization steps = 545
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [02:29<00:00, 149.08s/it]
***** Running evaluation *****


CPU times: user 16min 36s, sys: 1min 10s, total: 17min 46s
Wall time: 6min 8s


{'accuracy': 0.8150353178607467,
 'precision': 0.8146939923203654,
 'recall': 0.8150353178607467,
 'f1': 0.8148540954639961}